## Ví Dụ 3: Áp dụng BO để tìm ra cấu trúc mạng tốt nhất cho 1 mạng CNN

In [8]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import BayesianOptimization

Bộ dữ liệu CIFAR-10 gồm tổng cộng 60.000 hình ảnh màu (RGB), trong đó có 6.000 ảnh thuộc mỗi lớp. Bộ dữ liệu này đã được chia thành hai tập: tập huấn luyện (training set) với 50.000 hình ảnh và tập kiểm tra (test set) với 10.000 hình ảnh. Tập huấn luyện được sử dụng để đào tạo và điều chỉnh mô hình học máy, trong khi tập kiểm tra được sử dụng để đánh giá hiệu suất và khả năng tổng quát hóa của mô hình.
Các vật thể trong Bộ:

Airplane (Máy bay): Hình ảnh các máy bay từ các góc nhìn khác nhau.

Automobile (Ô tô): Hình ảnh các ô tô, xe tải, xe hơi từ các góc nhìn khác nhau.

Bird (Chim): Hình ảnh các loại chim từ các góc nhìn khác nhau.

Cat (Mèo): Hình ảnh các con mèo từ các góc nhìn khác nhau.

Deer (Hươu): Hình ảnh các con hươu từ các góc nhìn khác nhau.

Dog (Chó): Hình ảnh các con chó từ các góc nhìn khác nhau.

Frog (Ếch): Hình ảnh các con ếch từ các góc nhìn khác nhau.

Horse (Ngựa): Hình ảnh các con ngựa từ các góc nhìn khác nhau.

Ship (Tàu): Hình ảnh các loại tàu từ các góc nhìn khác nhau.

Truck (Xe tải): Hình ảnh các xe tải từ các góc nhìn khác nhau.


In [10]:
# Tải và chuẩn bị dữ liệu CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [11]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=(32, 32, 3)))

    # Thêm các khối convolutional với số lượng và số lượng bộ lọc được tối ưu hóa bởi BO
    for i in range(hp.Int("conv_blocks", min_value=1, max_value=3, step=1)):
        model.add(layers.Conv2D(filters=hp.Int(f"filters_{i}", min_value=16, max_value=64, step=16),
                                kernel_size=(3, 3), padding="same", activation="relu"))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int("dense_units", min_value=32, max_value=128, step=32),
                           activation="relu"))

    model.add(layers.Dense(10, activation="softmax"))

    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")),
        loss="categorical_crossentropy", metrics=["accuracy"])

    return model

In [12]:


# Tạo danh sách để lưu các giá trị
learning_rates = []
val_losses = []
val_accuracies = []

# Định nghĩa callback để lưu các giá trị sau mỗi epoch
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        learning_rate = self.model.optimizer.lr.numpy()
        val_loss = logs["val_loss"]
        val_accuracy = logs["val_accuracy"]
        
        learning_rates.append(learning_rate)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)


# Khởi tạo một đối tượng BayesianOptimization với hàm xây dựng mô hình và các thông số khác
tuner = BayesianOptimization(build_model,
                             objective="val_accuracy",
                             max_trials=10,
                             executions_per_trial=2,
                             directory="bayesian_optimization",
                             project_name="cnn_cifar10")

# Tìm kiếm các tham số tốt nhất cho mô hình và sử dụng callback để lưu giá trị
tuner.search(x_train, y_train,
             epochs=5,
             validation_data=(x_test, y_test),
             callbacks=[CustomCallback()])

# Lấy mô hình tốt nhất và đánh giá trên tập kiểm tra
best_model = tuner.get_best_models(num_models=1)[0]
best_model.evaluate(x_test, y_test)

# In ra learning rate và các chỉ số đánh giá sau mỗi epoch
for epoch in range(len(learning_rates)):
    print("Epoch:", epoch+1)
    print("Learning rate:", learning_rates[epoch])
    print("Validation loss:", val_losses[epoch])
    print("Validation accuracy:", val_accuracies[epoch])
    print()


Trial 10 Complete [00h 24m 34s]
val_accuracy: 0.6865499913692474

Best val_accuracy So Far: 0.6895999908447266
Total elapsed time: 01h 59m 02s
313/313 [==============================] - 8s 23ms/step - loss: 0.8952 - accuracy: 0.6914
Epoch: 1
Learning rate: 0.0010882807
Validation loss: 1.3241212368011475
Validation accuracy: 0.5284000039100647

Epoch: 2
Learning rate: 0.0010882807
Validation loss: 1.1664624214172363
Validation accuracy: 0.5871999859809875

Epoch: 3
Learning rate: 0.0010882807
Validation loss: 1.0525225400924683
Validation accuracy: 0.6312999725341797

Epoch: 4
Learning rate: 0.0010882807
Validation loss: 1.034347653388977
Validation accuracy: 0.6388999819755554

Epoch: 5
Learning rate: 0.0010882807
Validation loss: 0.9923528432846069
Validation accuracy: 0.6542999744415283

Epoch: 6
Learning rate: 0.0010882807
Validation loss: 1.3003382682800293
Validation accuracy: 0.5338000059127808

Epoch: 7
Learning rate: 0.0010882807
Validation loss: 1.1270145177841187
Validation 